In [12]:
import os
import traceback

In [13]:
import numpy as np
import cv2 
import imutils
from imutils import contours
from imutils.perspective import four_point_transform
from skimage.filters import threshold_local

In [6]:
pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


In [14]:
from PIL import Image
import pytesseract

In [29]:
pytesseract.pytesseract.tesseract_cmd = 'C:/Users/dipasha/AppData/Local/Programs/Tesseract-OCR/tesseract.exe'

input_directory = 'C:/Users/dipasha/Bank Guarantees'
output_directory = 'C:/Users/dipasha/Bank Guarantees Text'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# List all image files in the input directory
image_files = [f for f in os.listdir(input_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for image_file in image_files:
    input_path = os.path.join(input_directory, image_file)
    output_path = os.path.join(output_directory, os.path.splitext(image_file)[0])

    try:
        # reading file with cv2
        img = cv2.imread(input_path)

        if img is None:
            raise Exception(f"Unable to read the image at {input_path}")

        ratio = img.shape[0] / 500.0
        original_img = img.copy()

        # Additional preprocessing
        # Resize the image
        img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply histogram equalization for better contrast
        gray = cv2.equalizeHist(gray)

        # Apply adaptive thresholding
        _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Noise reduction using morphological operations
        kernel = np.ones((5, 5), np.uint8)
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

        # Find contours
        cnts, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw contours on a black background
        contour_img = np.zeros_like(img)
        cv2.drawContours(contour_img, cnts, -1, (255, 255, 255), thickness=cv2.FILLED)

        # Resize contour image to match the size of the original image
        contour_img = cv2.resize(contour_img, (original_img.shape[1], original_img.shape[0]))

        # Bitwise-AND to preserve only the regions inside the contours
        result_img = cv2.bitwise_and(original_img, contour_img)

        # Save the processed image
        processed_file_name = output_path + "-Processed.png"
        cv2.imwrite(processed_file_name, result_img)

        # Display the processed image
        cv2.imshow("Processed Image", result_img)
        cv2.waitKey(0)

        # Fetching text from the image and storing it into a text file
        file_text = pytesseract.image_to_string(Image.fromarray(result_img))
        text_file_name = output_path + "-Scanned.txt"
        with open(text_file_name, "a") as f:
            f.write(file_text + "\n")

    except Exception as e:
        print(f"Error processing image {input_path}: {str(e)}")
        continue  # Skip to the next iteration if an exception occurs

    #print(f"Processed image saved to: {scanned_file_name}")
    #print(f"Text file saved to: {text_file_name}")


Error processing image C:/Users/dipasha/Bank Guarantees\BCB-€1B-Cash-Funds-File-RWA-Letter-MT760-Verbiage-Fraud2-Example1.png: Unable to read the image at C:/Users/dipasha/Bank Guarantees\BCB-€1B-Cash-Funds-File-RWA-Letter-MT760-Verbiage-Fraud2-Example1.png
